In [5]:
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
import numpy as np
from PIL import Image
from sqlalchemy import create_engine, MetaData, Column, Integer, String, Enum, TIMESTAMP, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base
import enum
import os
import pickle
import datetime

In [6]:
# Database setup
Base = declarative_base()

class User(Base):
    __tablename__ = 'Users'
    user_id = Column(Integer, primary_key=True, autoincrement=True)
    username = Column(String(50), nullable=False)  
    email = Column(String(100), unique=True, nullable=False)  
    created_at = Column(TIMESTAMP, default=datetime.datetime.utcnow)

class FaceRecognition(Base):
    __tablename__ = 'FaceRecognition'
    recognition_id = Column(Integer, primary_key=True, autoincrement=True)
    user_id = Column(Integer, ForeignKey('Users.user_id'))
    image_path = Column(String(255), nullable=False)  
    detected_at = Column(TIMESTAMP, default=datetime.datetime.utcnow)
    user = relationship("User", back_populates="face_recognitions")
    fatigue_analyses = relationship("FatigueAnalysis", back_populates="recognition")

class FatigueStatus(enum.Enum):
    active = "active"
    fatigue = "fatigue"

class FatigueAnalysis(Base):
    __tablename__ = 'FatigueAnalysis'
    analysis_id = Column(Integer, primary_key=True, autoincrement=True)
    user_id = Column(Integer, ForeignKey('Users.user_id'))
    recognition_id = Column(Integer, ForeignKey('FaceRecognition.recognition_id'))
    status = Column(Enum(FatigueStatus), nullable=False)
    analyzed_at = Column(TIMESTAMP, default=datetime.datetime.utcnow)
    user = relationship("User", back_populates="fatigue_analyses")
    recognition = relationship("FaceRecognition", back_populates="fatigue_analyses")

class Log(Base):
    __tablename__ = 'Logs'
    log_id = Column(Integer, primary_key=True, autoincrement=True)
    user_id = Column(Integer, ForeignKey('Users.user_id'))
    message = Column(String(500), nullable=False)  
    logged_at = Column(TIMESTAMP, default=datetime.datetime.utcnow)
    user = relationship("User", back_populates="logs")

C:\Users\HSM\AppData\Local\Temp\ipykernel_20112\304263840.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [7]:
User.face_recognitions = relationship("FaceRecognition", order_by=FaceRecognition.recognition_id, back_populates="user")
User.fatigue_analyses = relationship("FatigueAnalysis", order_by=FatigueAnalysis.analysis_id, back_populates="user")
User.logs = relationship("Log", order_by=Log.log_id, back_populates="user")

In [8]:
# Database connection
# Fetch environment variables
db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_PASS")
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")

# Define the database connection URL
engine = create_engine(f'mysql+pymysql://{os.getenv("DB_USER")}:{os.getenv("DB_PASS")}@{os.getenv("DB_HOST")}/{os.getenv("DB_NAME")}')

# Create all tables defined by Base's subclasses
Base.metadata.create_all(engine)

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'None' ([Errno 11001] getaddrinfo failed)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [18]:
# Creating a session
Session = sessionmaker(bind=engine)
session = Session()

In [19]:
# Load the pre-trained YOLOv5 model for face detection
model_face = torch.hub.load('ultralytics/yolov5', 'yolov5s') 

Using cache found in C:\Users\HSM/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-5-31 Python-3.11.7 torch-2.3.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [20]:
# Load the trained fatigue detection model
class_names = ['active', 'fatigue']
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
fatigue_model = models.resnet18()
num_ftrs = fatigue_model.fc.in_features
fatigue_model.fc = nn.Linear(num_ftrs, 2)
fatigue_model.load_state_dict(torch.load('fatigue_detection_model.pth'))
fatigue_model = fatigue_model.to(device)
fatigue_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [21]:
# Define image pre-processing transformations
preprocess = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [26]:
# Function to detect face and predict fatigue
def detect_and_predict(frame):
    results = model_face(frame)

    # Debug: Print YOLOv5 results
    print(f"YOLOv5 Results: {results.xyxy[0]}")
    
    predictions = []
    
    for *box, conf, cls in results.xyxy[0]:
        x1, y1, x2, y2 = map(int, box)
        face = frame[y1:y2, x1:x2]

        # Debug: Print the detected face shape
        print(f"Detected Face Shape: {face.shape}")

        # Pre-process the face image
        face_pil = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face_pil = Image.fromarray(face_pil)
        face_tensor = preprocess(face_pil).unsqueeze(0).to(device)

        # Predict fatigue state
        with torch.no_grad():
            outputs = fatigue_model(face_tensor)
            _, preds = torch.max(outputs, 1)
            pred_class = class_names[preds[0]]

        predictions.append((x1, y1, x2, y2, pred_class))
    
    return predictions

In [27]:
# Function to register a new user
def register_user():
    username = input("Enter your name: ")
    email = input("Enter your email: ")

    # Check if the email already exists in the database
    if session.query(User).filter_by(email=email).first():
        print("This email is already registered. Please use a different email.")
        return False
        
    new_user = User(username=username, email=email)
    session.add(new_user)
    try:
        session.commit()
    except IntegrityError as e:
        print("This email is already registered. Please use a different email.")
        session.rollback()  # Rollback the session to avoid any corrupt state
        return False

    user_id = new_user.user_id

    # Capture face for recognition
    cap = cv2.VideoCapture(0) # Use either 0 or 1 for using integrated webcam or usb camera
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        cv2.imshow('Face Registration', frame)
        if cv2.waitKey(1) & 0xFF == ord('s'):  # Press 's' to save the image
            image_path = f"registered_faces/{username}.jpg"
            cv2.imwrite(image_path, frame)
            new_recognition = FaceRecognition(user_id=user_id, image_path=image_path)
            session.add(new_recognition)
            session.commit()
            break

    cap.release()
    cv2.destroyAllWindows()
    print("Registration complete.")

In [28]:
# Function to check if user is registered
def is_registered():
    email = input("Enter your email to check registration: ")
    user = session.query(User).filter_by(email=email).first()
    return user

In [30]:
# Main program
if __name__ == "__main__":
    if not os.path.exists('registered_faces'):
        os.makedirs('registered_faces')

    while True:
        response = input("Have you registered yet? (yes/no): ").strip().lower()
        if response == 'no':
            register_user()
        elif response == 'yes':
            user = is_registered()
            if user:
                break
            else:
                print("User not found. Please register first.")
        else:
            print("Please answer 'yes' or 'no'.")

    # Proceed with fatigue detection
    print(f"Hello, {user.username}!")

    # Open a video stream
    cap = cv2.VideoCapture(0)

    cv2.namedWindow('Fatigue Detection', cv2.WINDOW_NORMAL)  # Create a resizable window
    cv2.resizeWindow('Fatigue Detection', 640, 480)  # Set the window size

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        predictions = detect_and_predict(frame)

        # Draw bounding boxes and labels on the frame
        for (x1, y1, x2, y2, pred_class) in predictions:
            color = (0, 255, 0) if pred_class == 'active' else (0, 0, 255)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, pred_class, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        cv2.imshow('Fatigue Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to close the windows
            break

    cap.release()
    cv2.destroyAllWindows()

Have you registered yet? (yes/no):  yes
Enter your email to check registration:  giroud@milan.com


Hello, Giroud!
YOLOv5 Results: tensor([[ 16.82022,  99.87691, 519.28699, 478.34863,   0.60801,   0.00000]])
Detected Face Shape: (379, 503, 3)
YOLOv5 Results: tensor([[ 25.19792, 104.06999, 526.54907, 478.24426,   0.60312,   0.00000]])
Detected Face Shape: (374, 501, 3)
YOLOv5 Results: tensor([[4.72462e+01, 1.04225e+02, 4.43707e+02, 4.80000e+02, 4.58498e-01, 0.00000e+00]])
Detected Face Shape: (376, 396, 3)
YOLOv5 Results: tensor([[1.61812e+01, 1.05307e+02, 5.13814e+02, 4.77063e+02, 7.67808e-01, 0.00000e+00],
        [5.25459e+02, 3.69862e+00, 6.39286e+02, 3.59358e+02, 2.80619e-01, 0.00000e+00]])
Detected Face Shape: (372, 497, 3)
Detected Face Shape: (356, 114, 3)
YOLOv5 Results: tensor([[1.02485e+01, 1.03511e+02, 5.19388e+02, 4.78398e+02, 7.29905e-01, 0.00000e+00],
        [5.25295e+02, 1.78285e+00, 6.39317e+02, 3.55833e+02, 4.33277e-01, 0.00000e+00]])
Detected Face Shape: (375, 509, 3)
Detected Face Shape: (354, 114, 3)
YOLOv5 Results: tensor([[  6.74156, 107.92242, 513.52075, 476.9